# Score propagation
## Contenuti del notebook
* Si prendono 50 articoli: 25 da Breitbart e 25 da NYTimes. Si assegna un punteggio manuale: 0 se l'articolo è considerato oggettivo e 1 se considerato soggettivo.
* Si testano alcune tecniche per propagare la soggettività dalle parole all'articolo.
* Si osserva come i punteggi se la cavano con i benchmark.

In [1]:
#DO NOT EXECUTE THIS CELL
import pandas as pd
import glob
import os
import json
import numpy as np
import string
from ast import literal_eval
from jupyterthemes import jtplot
# currently installed theme will be used to
# set plot style if no arguments provided
jtplot.style()


#Retrieving the csv file with the articles.
#commenting this lines to prevent a mental breakdown

#articles = pd.read_csv("../data_safe.csv")
#articles["Paragraphs"] = articles["Paragraphs"].apply(literal_eval)

#sampling 25 articles from nytimes
#NY_Times_df = articles[articles["Newssite"] == "New York Times"].sample(25)
#sampling 25 articles from breitbart
#Breitbart = articles[articles["Newssite"] == "Breitbart"].sample(25)

In [2]:
### TIME TO READ SOME FUCKING ARTICLES
#NY_Times_df.to_csv("../benchmarks/NY_times.csv")
#Breitbart.to_csv("../benchmarks/Breitbart.csv")
import pandas as pd
import glob
import os
import json
import numpy as np
import string
from ast import literal_eval
from jupyterthemes import jtplot
NY_Times_df = pd.read_csv("../benchmarks/NY_times.csv", index_col = 0)
Breitbart = pd.read_csv("../benchmarks/Breitbart.csv", index_col=0)
Breitbart["Paragraphs"] = Breitbart["Paragraphs"].apply(literal_eval)
NY_Times_df["Paragraphs"] = NY_Times_df["Paragraphs"].apply(literal_eval)
words = ["said", "says", "stated", "states", "wrote", 
         "writes", "claimed", "claims", "reported", "reports", 
         "tweeted", "affirmed", "affirms", "tells", "told"]
scores_NY = [0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1,
             1, 1, 0]
scores_B = [0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1]

scores_NY_2 = [1, -1, 0, -1, -1, 1, 0, 0, -1, 1, 1, 0, 1, 1, 0, -1, -1, -1, 1, 0, 0, 0, 1, -1, 1]
scores_B_2 = []
NY_Times_df["Paragraphs"].iloc[15]

['President Trump revealed on Monday that he had been taking an anti-malaria drug as a preventive measure against the coronavirus, the same medicine that he has been promoting for two months with scant evidence of its efficacy and despite several warnings of dangerous side effects.',
 'The drug, hydroxychloroquine, has been invoked by Mr. Trump repeatedly since March during White House briefings on the coronavirus pandemic despite the reservations of doctors and scientists, including some advising the president. He even called the drug, which has been promoted by some conservative pundits, a “game changer.”',
 'Mr. Trump said he started taking the once-a-day pill about a week and a half ago. It was not immediately clear if Mr. Trump began the regimen in response to two White House staff members’ testing positive for the virus — one of the president’s personal valets and Katie Miller, the spokeswoman for Vice President Mike Pence.',
 'And last week, a federal agency head who had been in

In [3]:
#adding the lables and saving
Breitbart["benchmarks"] = scores_B
NY_Times_df["benchmarks"] = scores_NY
NY_Times_df["benchmarks_2"] = scores_NY_2
Breitbart.to_csv("../benchmarks/Breitbart.csv")
NY_Times_df.to_csv("../benchmarks/NY_times.csv")

# PARAGRAPHS BENCHMARKING

In [4]:
NY_Times_df
paragraphs = []
paragraphs = [par for article in NY_Times_df["Paragraphs"] for par in article ]
n_nyt = len(paragraphs)
paragraphs.extend([par for article in Breitbart["Paragraphs"] for par in article ])
n_b = len(paragraphs) - n_nyt

nyt = ["nyt" for i in range(n_nyt)]
breit = ["breitbart" for i in range(n_b)]
nyt.extend(breit)
df_pars = pd.DataFrame({
                        "Paragraphs" : paragraphs,
                        "Newssite" : nyt
                      })

In [5]:
#SAMPLE
df_pars["Score"] = None
df_pars_sample = df_pars.sample(200)
for index, row in df_pars_sample.iterrows():
    print(row["Paragraphs"])
    score = input()
    row["Score"] = score

“The lives of every African American matter, and there’s no place for racism in our country. But I oppose the BLM political organization due to its radical ideas and Marxist foundations, which include defunding the police and eroding the nuclear family,” Loeffler said.
1
All of this early growth is possible because his operation is exceedingly well-funded, a consequence of Mr. Trump taking the unprecedented step of filing his re-election campaign the day of his inauguration. His campaign had $40 million in the bank as of the last fund-raising report, with a goal of raising $1 billion for the 2020 race. (By comparison, President Barack Obama held less than $2 million in the bank at this point in his re-election cycle.)
1
Campbell described his motivation for composing the bill. He said, “I’m an artist in my private life. I paint, make jewelry, and so on. I’ve always been concerned about an artist’s or craftsmen’s right under the First Amendment, so I was thinking of that and reflecting 

0
Bishop DeGrood defined “those at increased risk for severe illness” as those people aged 65 or older, or anyone with cancer, chronic kidney disease, chronic obstructive pulmonary disease, a weakened immune system from an organ transplant, obesity, serious heart conditions, sickle cell disease, or type two diabetes mellitus.
0
Sanger, the founder of the Planned Parenthood abortion mill, is not “problematic” because she once told an off-color joke or flew a Confederate flag. The woman was a monster, a vile racist and eugenicist who praised the Nazi’s sterilization tactics, including those suffering from “gongenital feeble-mindedness; schizophrenia, circular insanity; heredity epilepsy; hereditary chorea (Huntington’s); hereditary blindness or deafness; grave hereditary bodily deformity; and chronic alcoholism.”
1
On Tuesday, a cargo of ammonium nitrate stored in the port of Beirut exploded, causing immense damage to Lebanon’s capital. The blast killed at least 157 people, injured 5,000

-1
The Times has created a new page that captures a snapshot of the 2020 Democratic primary as it currently stands, with the latest poll numbers, fund-raising stats and even rankings for who’s getting the most media coverage.
0
Participation in the program comes with a range of benefits. Along with permission to remain in the country, recipients can also get work permits, and can obtain health insurance from employers who offer it.
0
The theater is also removing every second row of seats, opening more entrances and eliminating intermissions to prevent restroom lines. And all attendees will have to wear masks (though they needn’t be Comedy or Tragedy).
0
“It’s hard to argue we need a change economically as people are doing well,” said Noah Williams, director of the Center for Research on the Wisconsin Economy at the University of Wisconsin — Madison.
-1
However, the time coincided with a rough stretch in Ms. Warren’s campaign nationally; including a downturn in polling, the departure of

0
“We don’t give a fuck about them fireworks,” one demonstrator insisted on the megaphone as the show enthralled the other protesters and their supporters.
1
To chart his own vision for the country, Mr. Biden left his home in Wilmington, Del., to travel to Philadelphia. It is the city where the nation’s founding documents were crafted, where President Barack Obama gave his famous speech on race in 2008, and where Mr. Biden held his first large-scale rally of the 2020 campaign, promising to heal the soul of the country. It is now also a city rocked by protests and growing racial tensions.
-1
The cutoff date to submit polling and donor data will be June 13, the D.N.C. added, which is about two weeks before the first debate.
0
The former U.S. senator and vice president described the current moment as one of historical import, amid a health crisis over the coronavirus, a recession, a national reckoning on race and a battle over the environment.
-1
“Otherwise, we are sending our people into

1
The campaign letter to the Commission on Presidential Debates requested an earlier debate, calling the current debate calendar outdated.
-1
Ms. Marion, a county board member who tends a small farm with a bed-and-breakfast, drew statewide attention a few years ago as part of efforts to pass legislation that became known as the “Cookie Bill.” State law didn’t authorize bakers to sell their goods at farmers’ markets without commercial licenses, and she wanted to change that, but struggled to get anywhere with the Republican-controlled Legislature. Eventually, she was among a group that sued. A court ruling opened the way for such sales, but Ms. Marion saw the events as telling.
1
Nearly all states have already suspended in-person classes for the rest of the academic year, and even in states that haven’t, many districts have said spring is too soon to open the schoolhouse doors again.
0
The Democrat’s convention, themed “Uniting America,” will be mostly virtual. There will be no physical

1
But Mr. Sanders maintained that he is opposed to much of what Mr. Trump has stood for and that he will fight any intolerance that he might try to promote as president.
-1
Mr. Biden’s campaign advisers have engaged in fluid and evolving deliberations about how best to wade back into public activity as Delaware eases restrictions.
-1
_____________________
-1
Instead, he provided his well-worn rationale for a single-payer system, saying it would afford families comprehensive insurance coverage and “save lives.”
0
In normal times practicing Catholics are obliged to attend Mass on Sundays and “holy days of obligation” throughout the year, but were granted a dispensation during the coronavirus pandemic.
-1
Mr. Penn said they were being careful to keep things disinfected. But he was not wearing a mask, and only some workers and customers had them on. No one’s temperature was being taken. And while he said he was concerned about safety, he also wondered if the virus was as dangerous as had b

In [7]:
df_pars_sample.to_csv("../benchmarks/bench_pars.cdv")